In [4]:
import pandas as pd
import spacy
from collections import defaultdict, Counter
import re
import ast

# Load English language model in spaCy
nlp = spacy.load("en_core_web_sm")

In [17]:
# Function for text preprocessing 
def clean_text(text):

    # Check if text is a string (handles NaN values)
    if isinstance(text, str):
         text.lower()
    else:
        str(text).lower()  # Convert non-string values to lowercase strings
    
    # Remove special characters, punctuation, and extra whitespace
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenize the text using spaCy
    doc = nlp(text)
    
    # Lemmatize tokens and remove stopwords
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    
    return tokens

In [23]:
def clean_text_batch(texts):
    cleaned_texts = []
    
    for doc in nlp.pipe(texts, batch_size=128):
        # Initialize a list to store cleaned tokens for each document
        cleaned_tokens = []
        
        for token in doc:
            # Preprocess each token: convert to lowercase, remove special characters
            processed_token = re.sub(r'[^a-zA-Z0-9\s]', '', token.text.strip().lower())
            
            # Skip stop words and append cleaned tokens to the list
            if not token.is_stop and processed_token:
                cleaned_tokens.append(processed_token)
        
        # Join cleaned tokens to form the cleaned text for each document
        cleaned_text = ' '.join(cleaned_tokens)
        cleaned_texts.append(cleaned_text)
    
    return cleaned_texts

In [18]:
# Function to extract semantic relationships
def extract_relations(text):
    doc = nlp(text)
    relations = []
    for sent in doc.sents:
        for token in sent:
            if token.dep_ in ["nsubj", "ROOT", "dobj"]:  # Adjust based on what relationships you want
                # Append the relationship tuple (head, dep, child) to the list
                relations.append((token.head.text, token.dep_, token.text))
    return relations

# Create new features based on semantic relationships
def create_new_features(row):
    title_relations = extract_relations(row['title'])
    text_relations = extract_relations(row['selftext'])
    
    # Combine the relationships from title and text
    all_relations = title_relations + text_relations
    
    return all_relations


In [24]:
# Read the CSV file
df = pd.read_csv('mental_disorders_reddit.csv')

# Create a new DataFrame with specific columns
columns_needed = ['title', 'selftext', 'subreddit']
df_subset = df[columns_needed].copy()

# print(df_subset.head())

# Preprocess text columns (title and selftext)
df_subset['title'] = clean_text_batch(df_subset['title'].fillna('').tolist())
df_subset['selftext'] = clean_text_batch(df_subset['selftext'].fillna('').tolist())

# Display a preview the preprocessed DataFrame
print(df_subset.head())

# Apply the function to create a new feature 'semantic_relationships'
df_subset['semantic_relationships'] = df_subset.apply(create_new_features, axis=1)

# Display the updated DataFrame with the new feature
print(df_subset.head())

#convert df to csv for future analysis
df_subset.to_csv('SemanticsRel.csv', index=False)

                                 title  \
0  Life is so pointless without others   
1                           Cold rage?   
2                I don’t know who I am   
3              HELP! Opinions! Advice!   
4                                 help   

                                            selftext subreddit  
0  Does anyone else think the most important part...       BPD  
1  Hello fellow friends 😄\n\nI'm on the BPD spect...       BPD  
2  My [F20] bf [M20] told me today (after I said ...       BPD  
3  Okay, I’m about to open up about many things I...       BPD  
4                                          [removed]       BPD  


In [7]:
# Read the CSV file
df_svo = pd.read_csv('SemanticsRel.csv')

# Create a new DataFrame with specific columns
columns_needed = ['subreddit','semantic_relationships']
df_svo = df_svo[columns_needed].copy()

# Filter out rows with empty relationships
df_svo = df_svo[df_svo['semantic_relationships'].apply(len) > 0]

# Convert string representations to actual lists of tuples
df_svo['semantic_relationships'] = df_svo['semantic_relationships'].apply(ast.literal_eval)

print(df_svo.head())
# df_svo.to_csv('svo.csv', index=False)

  subreddit                             semantic_relationships
0       BPD  [(pointless, ROOT, pointless), (think, ROOT, t...
1       BPD  [(rage, ROOT, rage), (discouraged, nsubj, spec...
2       BPD  [(know, ROOT, know), (told, nsubj, m20), (told...
3       BPD  [(help, ROOT, help), (help, dobj, opinions), (...
4       BPD     [(help, ROOT, help), (removed, ROOT, removed)]


In [8]:
clean_data = []

for idx, row in df_svo.iterrows():
    subreddit = row['subreddit']
    relationships = row['semantic_relationships']
    
    for rel in relationships:
        if len(rel) == 3:  # Ensure the tuple has three elements
            word1, dep, word2 = rel
            clean_data.append({'Subreddit': subreddit, 'Word1': word1, 'Dependency': dep, 'Word2': word2})
        else:
            print(f"Issue with relationship: {rel}")

clean_df = pd.DataFrame(clean_data)
print(clean_df)

             Subreddit      Word1 Dependency         Word2
0                  BPD  pointless       ROOT     pointless
1                  BPD      think       ROOT         think
2                  BPD      think       dobj  relationship
3                  BPD        ask      nsubj     therapist
4                  BPD    imagine      nsubj         goals
...                ...        ...        ...           ...
9979456  mentalillness      think      nsubj           run
9979457  mentalillness     having       dobj         think
9979458  mentalillness       kill       dobj       uranium
9979459  mentalillness  construct       dobj            fu
9979460  mentalillness       fuck       ROOT          fuck

[9979461 rows x 4 columns]


In [14]:
# Grouping the DataFrame by 'Subreddit', 'Word1', 'Dependency', and 'Word2' and counting occurrences
grouped = clean_df.groupby(['Subreddit', 'Word1', 'Dependency', 'Word2']).size().reset_index(name='Occurrence')

# Filtering relationships occurring more than three times within each subreddit
common_relationships = grouped[grouped['Occurrence'] > 3]

# Grouping by 'Word1', 'Dependency', and 'Word2' to count unique occurrences
relationship_counts = common_relationships.groupby(['Word1', 'Dependency', 'Word2']).size().reset_index(name='Subreddit_Count')

# Filtering relationships occurring in more than one subreddit
multiple_subreddit_relationships = relationship_counts[relationship_counts['Subreddit_Count'] > 1]

print(multiple_subreddit_relationships)


# # Count occurrences of SVO relationships within each subreddit
# subreddit_counts = defaultdict(Counter)
# for idx, row in df_svo.iterrows():
#     subreddit = row['subreddit']
#     for svo in row['semantic_relationships']:
#         subreddit_counts[subreddit][svo] += 1

# # Filter SVO relationships with count > 3 within each subreddit
# frequent_relationships = {}
# for subreddit, svo_counter in subreddit_counts.items():
#     frequent_relationships[subreddit] = {
#         svo: count for svo, count in svo_counter.items() if count > 3
#     }

# # Display frequent relationships per subreddit
# for subreddit, relationships in frequent_relationships.items():
#     print(f"Subreddit: {subreddit}")
#     for svo, count in relationships.items():
#         print(f"SVO: {svo}, Count: {count}")

          Word1 Dependency    Word2  Subreddit_Count
1             1       ROOT        1                3
8            17       ROOT       17                2
10           18       ROOT       18                3
11          18f       ROOT      18f                2
13          19f       ROOT      19f                2
...         ...        ...      ...              ...
187055   zombie       ROOT   zombie                3
187056     zone       ROOT     zone                3
187058   zoning       ROOT   zoning                2
187059     zoom       ROOT     zoom                2
187061  zyprexa       ROOT  zyprexa                6

[76895 rows x 4 columns]
